# The Lotka-Volterra Equations

This Jupyter notebook should allow you to experiment with numerically solving the Lotka-Volterra equations using [SciPy](https://www.scipy.org/) and plotting the results using [Matplotlib](https://matplotlib.org/). Have a read through everything before you start running things.

---

We begin by importing useful libraries and initialising things for a Jupyter Notebook:

In [ ]:
import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt

%matplotlib notebook

### The differential equations

The Lotka-Volterra equations are two coupled differential equations for predator and prey populations, $y(t)$ and $z(t)$. We are using the symbol $t$ here, but it should be noted that this is really the scaled time, $u$, from the book.

The equations then become:

$$
\hspace{-100px}\begin{cases}
    \dfrac{\textrm{d}y}{\textrm{d}t} &\equiv y' &= zy - \beta y &= (z - \beta) y \\ \\
    \dfrac{\textrm{d}z}{\textrm{d}t} &\equiv z' &= z - zy &= (1 - y) z
\end{cases}
$$

which are the driving equations of the population dynamics, and we have simplified them as much as we can. In order to solve them, we have to model both $y$ and $z$ together. We can do this by defining a vector (represented as a list in Python) of the $y$ and $z$ values and numerically integrating this:

`populations = [y, z]`.

---

To do this, we should first define a function `population_derivatives(...)` to represent these two differential equations, which we can then integrate (given boundary conditions) to find the populations at any given time.

It needs to take a vector of the populations, the time, and the parameter $\beta$ ; it must return the vector of the derivatives at that point. Note that these equations have no explicit $t$ dependence, but the SciPy library function we're going to use to do the integration doesn't know this; it will pass $t$ in as the second argument to our function, so we have to just ignore it.

We'll denote the derivatives in Python with $y'$ as `y_prime` and $z'$ as `z_prime`.

In [ ]:
def population_derivatives(populations, t, beta):
    # Extract y and z from the list:
    y = populations[0]
    z = populations[1]

    # Evaluate the derivatives at this point:
    y_prime = (z - beta) * y
    z_prime = (1 - y) * z

    # Return the vector of derivatives (as a list):
    return [y_prime, z_prime]

### Integrating the equations

We can now attempt to integrate this function numerically, using a method from the SciPy library: `scipy.integrate.odeint(...)`. We provide this with the Python version of the mathematical function we want to integrate, in this case `population_derivatives(...)`; the initial conditions; the times we want to know the result at; and finally any arguments (beyond the populations and time) that the function we're integrating needs, in this case $\beta$.

This SciPy method returns the populations at each instant of time we asked for as a two-dimensional array with $y(t)$ in the first column and $z(t)$ in the second. We can then extract $y$ and $z$ and plot them.

We must also decide what times we want to know the populations at; for now $1000$ evenly distributed samples in the range $0 \le t \le 16$ is enough to clearly see the behaviour, remembering that the units of $t$ are somewhat arbitrary due to the scaling. The mathematical functions are defined for every point in time, but computers cannot cope with an infinite number of values; instead we sample our function at this fixed number of points. Too few, and the plot will look rough and unlike the real function; too many and our code will never be able to work out all of the values required and may crash.

In [ ]:
# Set the one parameter left after scaling:
beta = 0.7

# Set the initial conditions for y and z:
y_0 = 0.1
z_0 = 0.7

# Decide the times we want to know the values of y and z at:
t_min = 0
t_max = 16
number_of_samples = 1000  # how many points to sample the populations at for plotting
t = np.linspace(t_min, t_max, number_of_samples)  # points in time to evaluate populations at

# Set the initial populations:
initial_populations = np.array([y_0, z_0])

# Integrate the two differential equations at the same time:
# The function `odeint(...)` takes the function to integrate as its first argument,
# initial conditions second, the time array third,
# and then `args`, the additional arguments to population_derivatives(...)
solution_list = scipy.integrate.odeint(population_derivatives, initial_populations, t, args=(beta,))

# Unpack the solution vector into its component arrays:
y = solution_list[:, 0]  # extract column 0, the y values
z = solution_list[:, 1]  # extract column 1, the z values

# Plot y and z values against time on the same plot:
plt.figure()
plt.title("The Lotka-Volterra Equations")
plt.plot(t, y, color='r', label='y(t)')
plt.plot(t, z, color='k', label='z(t)')
plt.legend(loc='best')  # add a plot legend
plt.xlabel('t', style='italic')  # add x-axis label
plt.ylabel('y, z', style='italic')  # add y-axis label
plt.gca().set_xlim(t[0], t[-1])  # show only the time range we calculated
plt.gca().set_ylim(bottom=0)  # populations cannot be negative, so y,z > 0

# Display the result:
plt.tight_layout()
plt.show()

### Exploring the parameters

We started with some values, $\beta = 0.7$, $y_{0} = 0.1$, $z_{0} = 0.7$ that show the curves leading and following. Which of $y$ and $z$ is predator and which is prey? Instead of labelling them with letters, we could have called the Python variables `predator` and `prey`, with `initial_predator` and `initial_prey`, to be more clear. We could also change the labels on the plot to say which curve is which more clearly.

When you're running this notebook yourself for the first time, go to the menu at the top and click "Cell" -> "Run All". Look at the plot this produces above. To explore, you can change the parameters in the cell above and press Ctrl-Enter to re-run that cell and update the plot.

---

#### To try:

Consider now the case where there are no predators to start with; modify the initial conditions to reflect this, leaving $\beta$ and the other initial condition the same. What would you expect to happen? If you run the code, does the plot reflect this? Does the model used by the Lotka-Volterra equations really hold true in this case?

Try the case where there is no longer any prey; set the initial predator value back to the one above, and set the initial prey to zero. What do you expect to happen to the predator population? Does the plot reflect this?

What values of $y_{0}$ and $z_{0}$ are required for stable populations of both predator and prey? This was Ex 7.5.5c in the book. What plot would you expect for stable populations, and does this model correctly demonstrate it?

You can also save images of your plots, if you want, by adding "`plt.savefig('lotka-volterra.png')`" to the end of the code above, modifying the name of the file to something sensible.